In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from pymoo.algorithms.moo.nsga2 import NSGA2
from pymoo.operators.crossover.sbx import SBX
from pymoo.operators.mutation.pm import PM
from pymoo.operators.sampling.rnd import FloatRandomSampling
from pymoo.termination import get_termination
from pymoo.optimize import minimize
from pymoo.core.problem import Problem
from cam_class_optimization import CamGeneration
import pandas as pd
import plotly.express as px

In [ ]:
class multi_objective_mechanism_optimization(Problem):
    
    # When intializing get the mechanism structure which is being optimized
    # (not the initial positions as we are trying to find those)
    def __init__(self,target_peak_force,target_percentage_max):
        
        # decision variable structure
        # elements 0-7: gear ratios
        # elements 8-15: gear percentages
        # element 16: initial gear ratio
        xl = np.ones(17)
        xu = np.ones(17)
        xl[0:8] *= 0.05
        xu[0:8] *= 1.5
        xl[8:16] *= 0.01
        xu[8:16] *= 1
        xl[16] *= 0.25
        xu[16] *= 0.75

        # super().__init__(n_var = (gear_ratios.shape[1]) + 1, n_obj=4, xl=xl, xu=xu, elementwise_evaluation=True)
        super().__init__(n_var = 17, n_obj=4, xl=xl, xu=xu, elementwise_evaluation=True)
        
        # Store mechanism information for later 
        # self.gear_ratios = gear_ratios
        # self.initial_gear_ratio = initial_gear_ratio
        self.target_peak_force = target_peak_force
        self.target_percentage_max = target_percentage_max
        # self.percentages = percentages
        
    def _evaluate(self, x, out, *args, **kwargs):
        
        objective_dict = {1:np.zeros(x.shape[0]),2:np.zeros(x.shape[0]),3:np.zeros(x.shape[0]),4:np.zeros(x.shape[0])}
        for i,sub_x in enumerate(x):
            # x is an array of <pop_size> 1-d arrays of gear ratios, the overall gear ratio, and gear percentages
            gear_ratios = sub_x[:8] # grabs all of the gear ratios except for the last one
            percentages = sub_x[8:-1] # grabs the gear percentages
            percentages *= 2 * np.pi
            initial_gr = sub_x[-1]
            gear_ratios = np.vstack((gear_ratios,percentages))
            # percentages = self.percentages
            # gear_ratios = np.hstack((gear_ratios,self.percentages))
            # gear_ratios= gear_ratios.reshape((2,-1))
            # gear_ratios[1,0] = 0
            # gear_ratios[1, :] = 2 * np.pi * gear_ratios[1, :]
            
            filename = 'data/Knee-angle_Chugo_2006.csv'
            Cam = CamGeneration(gear_ratios.T, initial_gr, sit_angle = np.pi * 220 / 180) # create Cam object given input gear ratios
            max_radius = Cam.calculate_cam_radii() # get maximum radius from generated cams
            forces,percentages = Cam.plot_forces_percentage(filename, torque = False, stroke = 0.1, plot = False) # get forces from generated cams
            ideal_perc_ind = np.where(np.round(percentages,1) == 32.3)[0][0] 
            ideal_perc_force = forces[ideal_perc_ind] # get magnitude of force at the stance percentage where max force is ideally timed
            max_force = max(forces)
            max_force_perc = percentages[np.where(forces == max_force)[0][0]] # get stance percentage where max force actually occurs
            sitting_perc_ind = np.where(np.round(percentages,1) == 0)[0][0]
            sitting_force = forces[sitting_perc_ind] # get actual forces at 0% stance (sitting)    

            objective_dict[1][i] = max_radius
            objective_dict[2][i] = abs(max_force_perc - self.target_percentage_max)
            objective_dict[3][i] = abs(max_force - self.target_peak_force)
            objective_dict[4][i] = abs(sitting_force)

        out["F"] = [objective_dict[1],objective_dict[2],objective_dict[3],objective_dict[4]] # size, max_force_perc, max_force_diff, sitting_force

In [ ]:
# gr0 = np.array( [ [0.18, 0.18, 6.00, 0.50, 1.40, 1.70, 1.50, 0.90],
#                   [0.00, 0.08, 0.28, 0.40, 0.45, 0.50, 0.60, 0.66] ] )
# gr0[1, :] = 2 * np.pi * gr0[1, :] # convert from percentages into radians
# gear_percentages = np.array([0  , .08, 0.16, 0.24, 0.32, 0.4, 0.48, 0.66])

# initial_gear_ratio=0.5

pop_size = 50
generations = 1500

# set up an instance of our problem
problem = multi_objective_mechanism_optimization(target_peak_force = 175,
                                                target_percentage_max = 32.5)
# set up algorithm
algorithm = NSGA2(
    pop_size = pop_size,
    n_offsprings = pop_size,
    sampling = FloatRandomSampling(),
    crossover = SBX(prob = 0.9, eta = 15),
    mutation = PM(eta = 20),
    eliminate_duplicates = True
)
termination = get_termination("n_gen", generations)
results = minimize(problem,
                   algorithm,
                   termination,
                   verbose = True,
                   save_history = True,
                   seed = 3)
X = results.X # design space values (decision variables)
F = results.F # objective space values



In [ ]:
""" percent_array = np.zeros_like(X[:,:8])
for i, elem in enumerate(percent_array):
    percent_array[i] = gear_percentages
xl, xu = problem.bounds()

plt.figure(figsize=(7, 5))
colors = ['r','g','b','c','k','m','y','w']
for i in range(8):
    plt.scatter(percent_array[:,i], X[:, i], s=30, facecolors=colors[7-i], edgecolors=colors[i])
plt.xlim(0, 1)
plt.ylim(xl[1], xu[1])
plt.title("Design Space")
plt.xlabel("Stance Percentage")
plt.ylabel("Gear Ratio")
plt.show()

df_dict = {'Size':F[:,0],'ideal_m_f_perc_diff':F[:,1],'ideal_max_force_diff':F[:,2],'sitting_force':F[:,3]}
df = pd.DataFrame(data=df_dict)
fig = px.scatter_3d(df, x='ideal_m_f_perc_diff', y='ideal_max_force_diff', z='sitting_force', color='Size')
fig.show() """

In [ ]:
if len(results.X) <10:
    print("Less than 10 results!")
    for sub_x in results.X:
        gear_ratios = sub_x[:8] # grabs all of the gear ratios except for the last one
        percentages = sub_x[8:-1] # grabs the gear percentages
        percentages *= 2 * np.pi
        initial_gr = sub_x[-1]
        gear_ratios = np.vstack((gear_ratios,percentages))
        # percentages = self.percentages
        # gear_ratios = np.hstack((gear_ratios,self.percentages))
        # gear_ratios= gear_ratios.reshape((2,-1))
        # gear_ratios[1,0] = 0
        # gear_ratios[1, :] = 2 * np.pi * gear_ratios[1, :]
        
        Cam = CamGeneration(gear_ratios.T, initial_gr, sit_angle = np.pi * 220 / 180) # create Cam object given input gear ratios
        max_radius = Cam.calculate_cam_radii() # get maximum radius from generated cams
        filename = 'data/Knee-angle_Chugo_2006.csv'
        forces,percentages = Cam.plot_forces_percentage(filename, torque = False, stroke = 0.1, plot = False) # get forces from generated cams
else:
    print(f'There are {len(X)} results in the pareto front!')
    objective_limit = 50
    for i,sub_x in enumerate(X):
        in_range_flag = True
        for res in F[i]:
            if res > objective_limit:
                in_range_flag = False
        if in_range_flag:
            print(f'New solution! This is index {i}!')
            gear_ratios = sub_x[:8] # grabs all of the gear ratios except for the last one
            percentages = sub_x[8:-1] # grabs the gear percentages
            percentages *= 2 * np.pi
            initial_gr = sub_x[-1]
            gear_ratios = np.vstack((gear_ratios,percentages))

            Cam = CamGeneration(gear_ratios.T, initial_gr, sit_angle = np.pi * 220 / 180)
            max_radius = Cam.calculate_cam_radii()
            inner_pts, outer_pts = Cam.cam_pts(stroke = 0.1)
            filename = 'data/Knee-angle_Chugo_2006.csv'
            forces, percentages = Cam.plot_forces_percentage(filename, torque = False, stroke = 0.1,plot=True)
            ideal_perc_ind = np.where(np.round(percentages,1) == 32.3)[0][0]
            ideal_perc_force = forces[ideal_perc_ind]
            max_force = max(forces)
            max_force_perc = percentages[np.where(forces == max_force)[0][0]]
            sitting_perc_ind = np.where(np.round(percentages,1) == 0)[0][0]
            sitting_force = forces[sitting_perc_ind]             

            # print("Force value at 32.3% is: ", ideal_perc_force)
            print('force at 32.3% = ',ideal_perc_force, '; max force of', max_force,' at ',max_force_perc, ' %')
            print('max radii sum = ',max_radius)
            print(f'sitting force = {sitting_force}')
            print(f'Max radius = {F[i][0]}, max_force_perc - 32.5 = {F[i][1]}, max_force_diff = {F[i][2]}, sitting_force = {F[i][3]}')
            print('TOTAL ERROR = ', F[i][0]+F[i][1]+F[i][2]+F[i][3])
            print('----------------------------------END----------------------------------')

In [ ]:
flag_array = np.zeros(np.shape(F[:,0]))
for i,sub_x in enumerate(F):
        for res in F[i]:
            if res > objective_limit:
                flag_array[i] = 1
Fvalid = np.delete(F, np.where(flag_array == 1), axis=0)
df_dict = {'Size':Fvalid[:,0],'ideal_m_f_perc_diff':Fvalid[:,1],'ideal_max_force_diff':Fvalid[:,2],'sitting_force':Fvalid[:,3]}
df = pd.DataFrame(data=df_dict)
fig = px.scatter_3d(df, x='ideal_m_f_perc_diff', y='ideal_max_force_diff', z='sitting_force',
            color='Size')
fig.show()